<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/MLP-LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(len(test.columns)-1, 1), activation='tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(Dense(20, activation='ReLU'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='ReLU'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 70, 100)           40800     
                                                                 
 dense_2 (Dense)             (None, 70, 100)           10100     
                                                                 
 gru_2 (GRU)                 (None, 100)               60600     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 111,601
Trainable params: 111,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 51s 8ms/step - loss: 9.5445e-04 - accuracy: 0.9998


In [ ]:
test_y = test.pop('Label')
test_x = test

In [ ]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 374s 5ms/step


In [ ]:
confusion_matrix(test_y, pred_y)

array([[1254671,    1070],
       [    596, 1292951]])

In [ ]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.99953   0.99915   0.99934   1255741
           1    0.99917   0.99954   0.99936   1293547

    accuracy                        0.99935   2549288
   macro avg    0.99935   0.99934   0.99935   2549288
weighted avg    0.99935   0.99935   0.99935   2549288

